# Example of orderbook simulation - Zero intelligence agents

In this notebook I utilise the zero-intelligence framework where market participants submit random market and limit orders according to pre-configured distributions, regardless of market conditions.

**IMPORTANT**: To ensure that the modules are discoverable, you should either run the notebook from the root of the repository *or* add the module directory to the `PYTHONPATH` variable on your machine.

In [1]:
import pandas as pd

from random import seed
import plotly.graph_objects as go
from datetime import datetime

from instrument import Instrument
from order_factory import OrderFactory
from orderbook import OrderBook
from visualisation import plot_orderbook

seed(42)

### Defining an imaginary *mVidia* instrument 

Creating `Instrument` and `OrderBook` objects

In [2]:
mvda = Instrument('MVDA', 'mVidia', True, min_tick_size=0.01)
display(mvda)

Instrument(code='MVDA', name='mVidia', is_active=True, min_tick_size=0.01)

In [3]:
mvda_orderbook = OrderBook(mvda)
display(mvda_orderbook.bids)
display(mvda_orderbook.asks)

deque([])

deque([])

### Pre-generating an initial state of the orderbook 

This is achieved by submitting a few limit orders first around the price of $100.00.

In [4]:
mvda_order_factory = OrderFactory(
    instrument=mvda,
    orderbook=mvda_orderbook,
    arrivals_rate=2.0,
    hazard_rate=0.01,
    buy_ratio=0.5,
    limit_order_ratio=1.0,
    min_consideration=1,
    limit_amount_lambda=1/3,
    market_amount_lambda=1/30,
    max_halfspread=1.0,
    static_midprice=100.0
)

In [5]:
# Generating 500 limit orders objects
start_time = datetime(2024, 1, 1)
limit_orders = mvda_order_factory.generate_orders(start_time=start_time, n_orders=500)
display(limit_orders[:5])

[Order(timestamp=datetime.datetime(2024, 1, 1, 0, 0, 0, 54068), cancellation_timestamp=datetime.datetime(2024, 1, 1, 0, 2, 15, 18588), counterpart_id=1010, instrument=Instrument(code='MVDA', name='mVidia', is_active=True, min_tick_size=0.01), order_type=<OrderType.LIMIT: 1>, side=<OrderSide.BUY: 1>, amount=1, price=99.99000000000001),
 Order(timestamp=datetime.datetime(2024, 1, 1, 0, 0, 0, 406026), cancellation_timestamp=datetime.datetime(2024, 1, 1, 0, 0, 3, 95466), counterpart_id=1008, instrument=Instrument(code='MVDA', name='mVidia', is_active=True, min_tick_size=0.01), order_type=<OrderType.LIMIT: 1>, side=<OrderSide.BUY: 1>, amount=1, price=99.99000000000001),
 Order(timestamp=datetime.datetime(2024, 1, 1, 0, 0, 0, 569023), cancellation_timestamp=datetime.datetime(2024, 1, 1, 0, 3, 24, 54334), counterpart_id=1003, instrument=Instrument(code='MVDA', name='mVidia', is_active=True, min_tick_size=0.01), order_type=<OrderType.LIMIT: 1>, side=<OrderSide.SELL: 2>, amount=2, price=100.06)

In [6]:
for i, lo in enumerate(limit_orders):
    mvda_orderbook.add_order(incoming_order=lo)
    # Cancelling orders after their lifetime expiry
    mvda_orderbook.perform_order_cancellations(lo.timestamp)

In [7]:
# The state of the orderbook after simulating 500 limit orders
plot_orderbook(mvda_orderbook, depth=10)

### Creating *mVidia* order factory 

Setting the following paramaters of the `OrderFactory` class:

- Arrival rate $\gamma = 2.05$, meaning that there will be on average 2.05 orders per second generated
- Hazard rate $\mu = 0.01$, meaning that the limit order lifetime on average will be around ~ 100 seconds  
- Buy ratio $\beta = 0.5$, we should expect an equal amount of buy and sell orders
- Limit ratio $\rho = \frac{2.0}{2.05} \approx 0.9755$, in conjunction with the value of $\gamma$, I should expect 2 limit orders per second and 1 market order every 20 seconds
- Lambda of limit order amounts $\lambda_L = \frac{1}{3}$, on average the size of the limit order will be $3.0$
- Lambda of market order amounts $\lambda_M = \frac{1}{30}$, on average the size of the limit order will be $30.0$
- Maximum halfspread $\sigma = 0.50$, the limit orders will be placed *at maximum* $0.50 away from the prevailing midpoint.

In [8]:
mvda_order_factory = OrderFactory(
    instrument=mvda,
    orderbook=mvda_orderbook,
    arrivals_rate=2.0 + 0.05,
    hazard_rate=0.01,
    buy_ratio=0.5,
    limit_order_ratio=2 / 2.05,
    min_consideration=1,
    limit_amount_lambda=1/3,
    market_amount_lambda=1/30,
    max_halfspread=1.0,
    static_midprice=None
)

### What are we expecting to see?

- Given the limit orders arrive at a rate of 2 orders / s and their average lifetime is 100, we should expect *on average* around 100 bid and 100 ask orders in orderbook at any given time
- Because of the $\lambda_L$ parameter within the exponential distribution and `np.ceil` rounding function, we expect an average LO amount of 3.5 - so the bids and asks should have on average $\approx 350$ volume of liquidity 
- Market orders arrive at the rate of 1 order / 20 seconds, hence about 5 times during average LO lifetime. Given the MO amount averages 30.5, they should consume about $30.5 \times 5 \times \frac{1}{2} = 76.25$ worth of volume from both the bid and ask side

The market order sizes should be significant enough to eat through the orderbook rungs, effectively moving the midprice, without consuming all available liquidity in the orderbook


In [9]:
from copy import deepcopy

orders_history = []
all_trades = []

# Starting trading from the last limit order timestamp of the previous simulation
current_time = limit_orders[-1].timestamp


# Generating 177,000 orders ~ 1 day of trading
for _ in range(177_000):
    new_order = mvda_order_factory.generate_order(previous_order_ts=current_time)
    orders_history.append(deepcopy(new_order))
    current_time = new_order.timestamp

    new_trades = mvda_orderbook.add_order(new_order)
    all_trades.extend(new_trades)
    # all_mids.append(aapl_orderbook.get_midprice())

    mvda_orderbook.perform_order_cancellations(current_time)

In [10]:
# The state of the orderbook after ~ 1 day of trading
plot_orderbook(mvda_orderbook, depth=10)

In [11]:
df_trades = pd.DataFrame(data=[(t.timestamp, t.price, t.amount) for t in all_trades], columns=['timestamp', 'price', 'amount']).set_index('timestamp')
df_resampled_trades = df_trades.resample('30s')['price'].agg(['first', 'max', 'min', 'last'])

fig = go.Figure(data=[go.Candlestick(x=df_resampled_trades.index,
                open=df_resampled_trades['first'],
                high=df_resampled_trades['max'],
                low=df_resampled_trades['min'],
                close=df_resampled_trades['last'])])

fig.update_layout(yaxis=dict(fixedrange=False), dragmode='zoom', xaxis_rangeslider_visible=False)
fig.show()